In [ ]:
import pandas as pd
import numpy as np

TRADING_DAYS = 252

def backtest_metrics(daily_returns: pd.Series) -> dict:
    """
    daily_returns: pd.Series of daily returns (e.g. 0.01 = 1%)
    """
    daily_returns = daily_returns.dropna()

    # --- Equity Curve ---
    equity = (1 + daily_returns).cumprod()

    # --- Total Return ---
    total_return = equity.iloc[-1] - 1

    # --- Annualized Return (CAGR) ---
    n_days = daily_returns.shape[0]
    annualized_return = equity.iloc[-1] ** (TRADING_DAYS / n_days) - 1

    # --- Volatility (annualized) ---
    volatility = daily_returns.std() * np.sqrt(TRADING_DAYS)

    # --- Sharpe Ratio ---
    sharpe = (
        daily_returns.mean() / daily_returns.std()
        * np.sqrt(TRADING_DAYS)
        if daily_returns.std() != 0 else np.nan
    )

    # --- Drawdown ---
    running_max = equity.cummax()
    drawdown = equity / running_max - 1

    max_dd = drawdown.min()

    # --- Max DD Peak / Recovery ---
    dd_bottom_date = drawdown.idxmin()
    dd_peak_date = equity.loc[:dd_bottom_date].idxmax()

    recovery_series = equity.loc[dd_bottom_date:]
    recovery_date = recovery_series[recovery_series >= equity.loc[dd_peak_date]].index
    recovery_date = recovery_date[0] if len(recovery_date) > 0 else None

    # --- Max DD Days ---
    if recovery_date is not None:
        max_dd_days = (recovery_date - dd_peak_date).days
    else:
        max_dd_days = np.nan

    # --- ATH Date ---
    ath_date = equity.idxmax()

    # --- Gain / Loss ratio ---
    gains = daily_returns[daily_returns > 0].mean()
    losses = -daily_returns[daily_returns < 0].mean()
    gain_loss = gains / losses if losses != 0 else np.nan

    return {
        "Total Return (%)": total_return * 100,
        "Annualized Return (%)": annualized_return * 100,
        "Volatility (%)": volatility * 100,
        "Sharpe Ratio": sharpe,
        "Max Drawdown (%)": max_dd * 100,
        "ATH Date": ath_date,
        "Max DD Peak": dd_peak_date,
        "Max DD Recovery": recovery_date,
        "Max DD Days": max_dd_days,
        "Gain / Loss": gain_loss
    }


In [53]:
df = pd.read_excel(r"C:\Users\0223c\OneDrive\文件\GitHub\investoraccessplatform\backtest\jamsret.xlsx", parse_dates=["Date"])
df.set_index("Date", inplace=True)
df = df[~((df["Daily Return"] == 0) & (df["SPY Return"] == 0))].loc['2025-01-01':'2025-12-31']
print(df)

jc1_returns = df["Daily Return"]      # JC1
sp500_returns = df["SPY Return"]       # S&P500

jc1_metrics = backtest_metrics(jc1_returns)
sp500_metrics = backtest_metrics(sp500_returns)

result = pd.DataFrame({
    "JC0": jc1_metrics,
    "S&P500": sp500_metrics
})

print(result)


            SPY Return  Daily Return
Date                                
2025-01-02     -0.0025       -0.0015
2025-01-03      0.0125        0.0024
2025-01-06      0.0058        0.0003
2025-01-07     -0.0113       -0.0117
2025-01-08      0.0015        0.0010
...                ...           ...
2025-12-25      0.0000       -0.0008
2025-12-26     -0.0001       -0.0012
2025-12-29     -0.0036       -0.0088
2025-12-30     -0.0012        0.0044
2025-12-31     -0.0074       -0.0050

[261 rows x 2 columns]
261
261
                                       JC0               S&P500
Total Return (%)                 26.802989            16.377789
Annualized Return (%)            25.768911            15.770716
Volatility (%)                    17.34882            18.985778
Sharpe Ratio                      1.408582              0.86504
Max Drawdown (%)                -12.991994           -19.014738
ATH Date               2025-12-04 00:00:00  2025-12-24 00:00:00
Max DD Peak            2025-02-18 00:00

In [51]:
df = pd.read_excel(r"C:\Users\0223c\OneDrive\文件\GitHub\investoraccessplatform\backtest\jc1ret.xlsx", parse_dates=["Date"])
df.set_index("Date", inplace=True)
df = df[~((df["Daily Return"] == 0) & (df["SPY Return"] == 0))].loc['2025-01-01':'2025-12-31']
print(df)

jc1_returns = df["Daily Return"]      # JC1
sp500_returns = df["SPY Return"]       # S&P500

jc1_metrics = backtest_metrics(jc1_returns)
sp500_metrics = backtest_metrics(sp500_returns)

result = pd.DataFrame({
    "JC1": jc1_metrics,
    "S&P500": sp500_metrics
})

print(result)

            SPY Return  Daily Return
Date                                
2025-05-21     -0.0169       -0.0004
2025-05-22      0.0004        0.0002
2025-05-23     -0.0068        0.0005
2025-05-27      0.0208        0.0006
2025-05-28     -0.0058       -0.0008
...                ...           ...
2025-12-25      0.0000       -0.0142
2025-12-26     -0.0001        0.0136
2025-12-29     -0.0036       -0.0071
2025-12-30     -0.0012       -0.0013
2025-12-31     -0.0074       -0.0033

[180 rows x 2 columns]
                                       JC1               S&P500
Total Return (%)                 22.412408            15.042938
Annualized Return (%)            32.935974            21.675728
Volatility (%)                    11.00642             10.07648
Sharpe Ratio                      2.642602              1.99792
Max Drawdown (%)                 -3.172675            -4.109918
ATH Date               2025-11-29 00:00:00  2025-12-24 00:00:00
Max DD Peak            2025-11-20 00:00:00  202

In [45]:
spy

Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
1993-01-29,24.241400,24.258641,24.137951,24.258641,1003200
1993-02-01,24.413824,24.413824,24.258651,24.258651,480500
1993-02-02,24.465548,24.482789,24.344858,24.396582,201300
1993-02-03,24.724163,24.741404,24.482783,24.500025,529400
1993-02-04,24.827618,24.879342,24.534514,24.810376,531500
...,...,...,...,...,...
2026-01-16,691.659973,694.250000,690.099976,693.659973,79289200
2026-01-20,677.580017,684.770020,676.570007,681.489990,111623300
